In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pdp
import sklearn
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE




/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
import pandas_profiling as pdp
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/Data

In [0]:

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [0]:
Labels = train_data['m13']
train_data.drop('m13',axis= 1,inplace= True)
#print(Labels)
#X=train_data['interest_rate','unpaid_principal_bal','loan_term','loan_to_value','number_of_borrowers','debt_to_income_ratio','borrower_credit_score','co-borrower_credit_score','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']

In [0]:
def responseTime(train_data,k):
  l=[]
  m=[]
  responseTime = []
  if(k==0):
    for i in train_data['first_payment_date']:
      l.append(int(i.split('/')[0]))
    for j in train_data['origination_date']:
      m.append(int(j.split('-')[1]))
  if(k==1):
    for i in train_data['first_payment_date']:
      l.append(int(i.split('-')[1]))
    for j in train_data['origination_date']:
      m.append(int(j.split('/')[0]))
  for k in l:
    for n in m:
      responseTime= k-n
  return responseTime

In [0]:
def mdf(train_data):
  cols=['val','var0','var1','var2','var3','var4','var5','var6','var7','var8','var9','var10','var11']
  mdf=pd.DataFrame(columns=cols)
  mdf['val']=df['m1']-df['m12']

  for i in range(0,12):
    mdf['var0']= df['m1']-df['m2']
    mdf['var1']= df['m2']-df['m3']
    mdf['var2']= df['m3']-df['m4']
    mdf['var3'] = df['m4']-df['m5']
    mdf['var4']= df['m5']-df['m6']
    mdf['var5']= df['m6']-df['m7']
    mdf['var6']= df['m7']-df['m8']
    mdf['var7']= df['m8']-df['m9']
    mdf['var8'] = df['m9']-df['m10']
    mdf['var9']= df['m10']-df['m11']
    mdf['var10']= df['m10']-df['m11']
    mdf['var11']=df['m11']-df['m12']
  mdf.fillna(0)
  mdf.replace(' ',0)
  return mdf

In [0]:
def preprocess(train_data,k):
    train_data['responseTime'] = responseTime(train_data,k)
    mdf=mdf(train_data)
    train_data.drop(columns = ['origination_date','first_payment_date'],axis =1,inplace = True)
    vectorizer = TfidfVectorizer(min_df=10)
    fi_tfidf = vectorizer.fit_transform(train_data['financial_institution'])
    train_data['source'] = train_data['source'].replace(to_replace=['X','Y','Z'],value=[0,1,2])
    print(train_data['source'])
    #source=  vectorizer.fit_transform(train_data['source'])
    #oan_purposse= vectorizer.fit_transform(train_data['loan_purpose'])

    print("Shape of matrix after one hot encodig ",fi_tfidf.shape)
    #rint("Shape of matrix after one hot encodig ",loan_purposse)

    dropcolus=['loan_purpose','financial_institution','loan_id','m1','m2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12']

    train_data.drop(dropcolus,axis = 1,inplace =True)
    print(train_data.columns)
    
    train_data.fillna(0)
    price_scalar = StandardScaler()
    price_scalar.fit(train_data.values.reshape(-1,1)) # finding the mean and standard deviation of this data
    print(f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")

    # Now standardize the data with above maen and variance
    if(k==0):
        price_standardized = price_scalar.transform(train_data.values.reshape(116058,24))
    else:
        price_standardized = price_scalar.transform(train_data.values.reshape(35866,24))
    X = hstack((fi_tfidf,price_standardized))
    X.shape
    X=pd.concat([X,mdf],axis=1)
    return X
# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)


In [0]:
X_train = preprocess(train_data,0)
X_test = preprocess(test_data,1)

# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)


In [0]:

# with the same hstack function we are concatinating a sparse matrix and a dense matirx :)


In [0]:



# def responseTime(df):
#       diff = []
#       for i in range(len(df['origination_date'])):
#         diff = int(df['first_payment_date'][i].split('/')[0]) - int(df['origination_date'][i].split('-')[1])
#         return diff
# test_data['responseTime'] = responseTime(test_data)
# test_data.drop(columns = ['origination_date','first_payment_date'],axis =1,inplace = True)

# # tsne = TSNE(n_components=2, perplexity=30, learning_rate=200)

# # X_embedding = tsne.fit_transform(X.toarray())
# # # if x is a sparse matrix you need to pass it as X_embedding = tsne.fit_transform(x.toarray()) , .toarray() will convert the sparse matrix into dense matrix

# # for_tsne = np.hstack((X_embedding, Labels.reshape(-1,1)))
# # for_tsne_df = pd.DataFrame(data=for_tsne, columns=['Dimension_x','Dimension_y','Score'])
# # colors = {0:'red', 1:'blue', 2:'green'}
# # plt.scatter(for_tsne_df['Dimension_x'], for_tsne_df['Dimension_y'], c=for_tsne_df['Score'].apply(lambda X: colors[x]))
# # plt.show()

In [0]:
import imblearn
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
X_train,Labels = sm.fit_sample(X_train,Labels)
# from sklearn import svm
# wclf = svm.SVC(kernel='linear', class_weight={1:10})
# wclf.fit(X, Labels)

In [0]:
# from sklearn import ensemble
# clf1 = ensemble.AdaBoostClassifier(n_estimators=1000)
# clf1.fit(train, labels)


In [0]:

clf =RandomForestClassifier()
params = {'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)]} 
gs = GridSearchCV(estimator=clf, param_grid=params, cv=5)
gs.fit(X_train, Labels)


In [0]:
gs.predict(X_test)

In [0]:
# y =gs.predict(X)
# sklearn.metrics.f1_score(Labels, y)

In [0]:

# vectorizer = TfidfVectorizer(min_df=10)
# fi_tfidf = vectorizer.fit_transform(test_data['financial_institution'])
# test_data['source'] = test_data['source'].replace(to_replace=['X','Y','Z'],value=[0,1,2])
# print(test_data['source'])
# #source=  vectorizer.fit_transform(train_data['source'])
# #oan_purposse= vectorizer.fit_transform(train_data['loan_purpose'])

# print("Shape of matrix after one hot encodig ",fi_tfidf.shape)
# #rint("Shape of matrix after one hot encodig ",loan_purposse)
# from sklearn.decomposition import PCA


In [0]:
# dropcolus=['loan_purpose','financial_institution','origination_date','first_payment_date','loan_id']
# z= test_data.drop(dropcolus,axis=1)


In [0]:
# # price_standardized = standardScalar.fit(project_data['price'].values)
# # this will rise the error
# # ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# # Reshape your data either using array.reshape(-1, 1)

# price_scalar = StandardScaler()
# price_scalar.fit(z.values.reshape(-1,1)) # finding the mean and standard deviation of this data
# print(f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")

# # Now standardize the data with above maen and variance.

# price_standardize = price_scalar.transform(z.values.reshape(35866,23 ))

In [0]:

# # with the same hstack function we are concatinating a sparse matrix and a dense matirx :)
# Z = hstack((fi_tfidf,price_standardize))
# Z.shape

In [0]:
# pred =gs.predict(Z)
# predictions = clf1.predict_proba(Z)[:,1]
# # for i in pred:
# #     if(i==1):
# #         print("Hurray")

In [0]:
def adaboost(X_train, X_test, y_train):
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred
# AdaBoost
y_baseline = adaboost(X_train, X_test, Labels)
# SMOTE
# sm = SMOTE(random_state=42)
# X_train_sm, Labels = sm.fit_sample(X_train, Labels)
# y_smote = adaboost(X_train_sm, X_test, Labels)
# # RUS
# X_full = X_train.copy()
# X_full['target'] = Labels[1]
# X_maj = X_full[X_full.target==0]
# X_min = X_full[X_full.target==1]
# X_maj_rus = resample(X_maj,replace=False,n_samples=len(X_min),random_state=44)
# X_rus = pd.concat([X_maj_rus, X_min])
# X_train_rus = X_rus.drop(['target'], axis=1)
# y_train_rus = X_rus.target
# y_rus = adaboost(X_train_rus, X_test, y_train_rus)

In [0]:
#sklearn.metrics.f1_score(Labels, X_test)
test_data1 = pd.read_csv("test.csv")
cols=['loan_id','m13']
output=pd.DataFrame(columns=cols)
print(output.columns)
output['loan_id']=test_data1['loan_id']
output['m13']=y_baseline
output.to_csv("pred1.csv", index=False, header=True)